In [27]:
from alpaca_trade_api.rest import REST, TimeFrame
import time

# ตั้งค่า API Key และ Secret สำหรับ Paper Account
API_KEY = "PK6JC4NIPDDSHGSSEMUX"  # ใส่ API Key ของคุณ
SECRET_KEY = "YwbBplZK6GLawJuoFm0g4MJbh0rY3JlTnx8kpzQa"  # ใส่ Secret Key ของคุณ
BASE_URL = "https://paper-api.alpaca.markets" # URL สำหรับ Paper Account

# สร้างการเชื่อมต่อกับ Alpaca
api = REST(API_KEY, SECRET_KEY, BASE_URL)

# ตรวจสอบสถานะบัญชี
account = api.get_account()
print(f"Account status: {account.status}")  # แสดงสถานะบัญชี


Account status: ACTIVE


In [2]:
# ฟังก์ชันสำหรับซื้อหุ้น
def buy_stock(symbol, qty):
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side='buy',
            type='market',
            time_in_force='gtc'  # Good Till Cancelled
        )
        print(f"Order placed to buy {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error buying stock: {e}")

# ฟังก์ชันสำหรับขายหุ้น
def sell_stock(symbol, qty):
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side='sell',
            type='market',
            time_in_force='gtc'
        )
        print(f"Order placed to sell {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error selling stock: {e}")


In [24]:
def trade_strategy(symbol, buy_price, sell_price):
    while True:
        # ดึงข้อมูลราคาหุ้นล่าสุด
        barset = api.get_bars(symbol, TimeFrame.Minute, limit=1)
        price=barset[0].c
        if barset:
            for bar in barset:
                print(f"Time: {bar.t}, Open: {bar.o}, High: {bar.h}, Low: {bar.l}, Close: {bar.c}, Volume: {bar.v}")

        # กลยุทธ์การซื้อขาย
        if price <= buy_price:
            print(f"Buying {symbol} at {price}")
            api.submit_order(
                symbol=symbol,
                qty=1,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
        elif price >= sell_price:
            print(f"Selling {symbol} at {price}")
            api.submit_order(
                symbol=symbol,
                qty=1,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
        time.sleep(1) 


In [31]:
# เริ่มต้นกลยุทธ์สำหรับหุ้น AAPL
trade_strategy(symbol="AAPL", buy_price=247.75, sell_price=250)


Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706


KeyboardInterrupt: 

In [15]:
symbol="AAPL"
barset = api.get_bars(symbol, TimeFrame.Minute, limit=1)
print(f"Raw data for {symbol}: {barset}")


Raw data for AAPL: []


In [30]:
import asyncio
from alpaca_trade_api.stream import Stream
import nest_asyncio
nest_asyncio.apply()



base_url = "https://paper-api.alpaca.markets"
data_url = "wss://stream.data.alpaca.markets/v2/sip"  # ใช้ SIP สำหรับข้อมูลเรียลไทม์

symbol = "AAPL"  # หุ้นที่ต้องการติดตาม

async def quote_handler(data):
    """แสดงข้อมูลราคาล่าสุด"""
    print(f"Time: {data.timestamp}, Symbol: {data.symbol}, Bid: {data.bid_price}, Ask: {data.ask_price}")

async def main():
    stream = Stream(api_key, api_secret, base_url=data_url, raw_data=True)
    stream.subscribe_quotes(quote_handler, symbol)  # สมัครรับข้อมูลเรียลไทม์
    await stream.run()

# ตรวจสอบว่ามี event loop อยู่แล้วหรือไม่
if __name__ == "__main__":
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # ไม่มี event loop ที่กำลังทำงาน
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

    # รันฟังก์ชัน main
    loop.run_until_complete(main())


error during websocket communication: auth failed
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 251, in _run_forever
    await self._start_ws()
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 105, in _start_ws
    await self._auth()
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 99, in _auth
    raise ValueError(msg[0].get('msg', 'auth failed'))
ValueError: auth failed
error during websocket communication: auth failed
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 251, in _run_forever
    await self._start_ws()
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 105, in _start_ws
    await self._auth()
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 99, in _auth
    raise ValueError(msg[0].get

keyboard interrupt, bye


/opt/anaconda3/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


TypeError: object NoneType can't be used in 'await' expression

In [21]:
import yfinance as yf

# กำหนดสัญลักษณ์หุ้น (ตัวอย่าง: AAPL - Apple)
symbol = "NVDA"

# ดึงข้อมูลย้อนหลัง 10 วัน
data = yf.download(symbol, period="11d", interval="1d")

# แสดงผลข้อมูล
print(data)



[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            NVDA        NVDA        NVDA        NVDA       NVDA
Date                                                                 
2025-01-16  133.570007  138.750000  133.490005  138.639999  209235600
2025-01-17  137.710007  138.500000  135.460007  136.690002  201188800
2025-01-21  140.830002  141.830002  137.089996  139.160004  197749000
2025-01-22  147.070007  147.789993  143.669998  144.660004  237651400
2025-01-23  147.220001  147.229996  143.720001  145.050003  155915500
2025-01-24  142.619995  148.970001  141.880005  148.369995  234657600
2025-01-27  118.419998  128.399994  116.699997  124.800003  818830900
2025-01-28  128.990005  129.000000  116.250000  121.809998  579666400
2025-01-29  123.699997  126.889999  120.050003  126.500000  467120600
2025-01-30  124.650002  125.000000  118.099998  123.099998  392925500
2025-01-31  120.070000  127.849998  119.190002  123.779999  388161100


In [17]:
import pandas as pd
pd.DataFrame(data)

Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2025-01-17,137.710007,138.500000,135.460007,136.690002,201188800
2025-01-21,140.830002,141.830002,137.089996,139.160004,197749000
2025-01-22,147.070007,147.789993,143.669998,144.660004,237651400
2025-01-23,147.220001,147.229996,143.720001,145.050003,155915500
2025-01-24,142.619995,148.970001,141.880005,148.369995,234657600
2025-01-27,118.419998,128.399994,116.699997,124.800003,818830900
2025-01-28,128.990005,129.000000,116.250000,121.809998,579666400
2025-01-29,123.699997,126.889999,120.050003,126.500000,467120600


In [22]:
data["Close"]["NVDA"]

Date
2025-01-16    133.570007
2025-01-17    137.710007
2025-01-21    140.830002
2025-01-22    147.070007
2025-01-23    147.220001
2025-01-24    142.619995
2025-01-27    118.419998
2025-01-28    128.990005
2025-01-29    123.699997
2025-01-30    124.650002
2025-01-31    120.070000
Name: NVDA, dtype: float64

In [15]:
import numpy as np
np.array(data["Close"]["NVDA"])

array([137.71000671, 140.83000183, 147.07000732, 147.22000122,
       142.61999512, 118.41999817, 128.99000549, 123.69999695,
       124.65000153, 120.06999969])

In [18]:
k=pd.DataFrame()

In [19]:
k["Close"]=np.array(data["Close"]["NVDA"])

In [20]:
k

,Close
0,137.710007
1,140.830002
2,147.070007
3,147.220001
4,142.619995
5,118.419998
6,128.990005
7,123.699997
8,124.650002
9,120.070000


In [1226]:
import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import yfinance as yf

# กำหนดสัญลักษณ์หุ้น (ตัวอย่าง: AAPL - Apple)
symbol = "NVDA"

# ดึงข้อมูลย้อนหลัง 10 วัน
data = yf.download(symbol, period="3000d", interval="1d")

df=pd.DataFrame()
df["Open"]=np.array(data["Open"]["NVDA"])
df["High"]=np.array(data["High"]["NVDA"])
df["Low"]=np.array(data["Low"]["NVDA"])
df["Close"]=np.array(data["Close"]["NVDA"])
df["Volume"]=np.array(data["Volume"]["NVDA"])

df["diff_n_o"]=[None]+[np.log(df["Open"][i+1])-np.log(df["Open"][i]) for i in (range(len(df)-1))]
df["diff_n_h"]=[None]+[np.log(df["High"][i+1])-np.log(df["High"][i]) for i in (range(len(df)-1))]
df["diff_n_c"]=[None]+[np.log(df["Close"][i+1])-np.log(df["Close"][i]) for i in (range(len(df)-1))]
df["diff_n_v"]=[None]+[np.log(df["Volume"][i+1])-np.log(df["Volume"][i]) for i in (range(len(df)-1))]
df["diff_n_l"]=[None]+[np.log(df["Low"][i+1])-np.log(df["Low"][i]) for i in (range(len(df)-1))]
df["diff_n_CO"]=[None]+[np.log(df["Close"][i+1])-np.log(df["Open"][i]) for i in (range(len(df)-1))]
data2=df.dropna()
Xl1=data2[["diff_n_o","diff_n_l","diff_n_h","diff_n_c","diff_n_v","diff_n_CO"]]

[*********************100%***********************]  1 of 1 completed


In [1062]:
ddd = yf.download(symbol, period="2d", interval="1m")

[*********************100%***********************]  1 of 1 completed


In [1064]:
ddd



Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Datetime,,,,,
2025-01-30 14:30:00+00:00,123.110001,124.150002,123.000000,123.080002,16459594
2025-01-30 14:31:00+00:00,123.315002,123.669998,122.959999,123.105003,1425718
2025-01-30 14:32:00+00:00,122.649902,123.690002,122.500000,123.330002,1751337
2025-01-30 14:33:00+00:00,122.425598,122.900002,122.250000,122.650002,1613022
2025-01-30 14:34:00+00:00,122.214996,122.500000,121.919998,122.447197,1725838
...,...,...,...,...,...
2025-01-31 20:55:00+00:00,120.179901,120.250000,119.989998,120.074997,1134160
2025-01-31 20:56:00+00:00,120.165001,120.279999,120.010002,120.190002,1802534


In [1048]:
np.array(data)[-1][3]

array([1.20070000e+02, 1.27849998e+02, 1.19190002e+02, 1.23779999e+02,
       3.88161100e+08])

In [1049]:
data

Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2013-03-01,0.295083,0.296012,0.289279,0.291833,389396000
2013-03-04,0.293922,0.296708,0.290672,0.292994,353680000
2013-03-05,0.297173,0.297637,0.294154,0.295547,366168000
2013-03-06,0.296940,0.301816,0.295547,0.299030,342928000
2013-03-07,0.296244,0.301584,0.294619,0.298566,314824000
...,...,...,...,...,...
2025-01-27,118.419998,128.399994,116.699997,124.800003,818830900
2025-01-28,128.990005,129.000000,116.250000,121.809998,579666400


In [1025]:
from keras.saving import register_keras_serializable

@register_keras_serializable(package="Custom", name="acc")
def acc(y_true, y_pred):
    YT = tf.cast(y_true > 0, tf.int8)
    YP = tf.cast(y_pred > 0, tf.int8)
    accuracy = tf.reduce_mean(tf.cast(YT == YP, tf.float32))
    return accuracy
# Load pre-trained RNN model
@st.cache_resource
def load_model():
    return tf.keras.models.load_model('AI.keras', custom_objects={
    'acc': acc,
})


In [1182]:

model = load_model()

In [64]:
predicted_price = model.predict(np.array(Xl1.values[np.newaxis]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step


In [31]:
np.array(Xl1.values)

array([[-1.41650350e-02,  1.46498335e-02, -1.80342700e-03,
         3.05243337e-02, -3.92171191e-02,  7.43446747e-03],
       [ 1.79087706e-02,  1.19611744e-02,  2.37588446e-02,
         2.24034287e-02, -1.72452209e-02,  1.19291256e-02],
       [ 3.87618101e-02,  4.68813725e-02,  4.11631316e-02,
         4.33552115e-02,  1.83806344e-01,  1.65225270e-02],
       [ 2.69234487e-03,  3.47980461e-04, -3.79634094e-03,
         1.01936121e-03, -4.21490700e-01,  1.48495433e-02],
       [ 2.26305894e-02, -1.28853064e-02,  1.17489907e-02,
        -3.17443586e-02,  4.08813237e-01, -3.95254047e-02],
       [-1.72996641e-01, -1.95375151e-01, -1.48594608e-01,
        -1.85946105e-01,  1.24975016e+00, -5.24748687e-02],
       [-2.42500467e-02, -3.86346867e-03,  4.66206064e-03,
         8.54973129e-02, -3.45424826e-01,  5.72724909e-02],
       [ 3.77798745e-02,  3.21653036e-02, -1.64918398e-02,
        -4.18756698e-02, -2.15865297e-01, -2.23830534e-02],
       [-2.72452874e-02, -1.63766378e-02, -1.500

In [34]:
predicted_price[0][0]

-0.0038487613

In [615]:
Xl1

,diff_n_o,diff_n_l,diff_n_h,diff_n_c,diff_n_v,diff_n_CO
1,0.003970,0.004804,0.002350,-0.003942,-0.096204,0.003165
2,0.008679,0.011910,0.003125,0.010998,0.034700,0.005484
3,0.011714,0.004725,0.013943,-0.000781,-0.065572,-0.007012
4,-0.001554,-0.003147,-0.000770,-0.002348,-0.085507,-0.007806
5,-0.005458,0.003147,-0.008504,0.004691,-0.244798,0.002343
...,...,...,...,...,...,...
2995,-0.172997,-0.195375,-0.148595,-0.185946,1.249750,-0.052475
2996,-0.024250,-0.003863,0.004662,0.085497,-0.345425,0.057272
2997,0.037780,0.032165,-0.016492,-0.041876,-0.215865,-0.022383
2998,-0.027245,-0.016377,-0.015007,0.007651,-0.172967,0.012513


In [37]:
import yfinance as yf

# ระบุสัญลักษณ์หุ้นและวันที่ที่ต้องการดึงข้อมูล
ticker = "NVDA"  # ตัวอย่างหุ้น Apple
date = "2024-11-8"

# ดึงข้อมูลหุ้น
stock = yf.download(ticker, start=date, end=date)

# แสดงผลลัพธ์
print(stock)


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NVDA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-11-8 -> 2024-11-8)')


Empty DataFrame
Columns: [(Adj Close, NVDA), (Close, NVDA), (High, NVDA), (Low, NVDA), (Open, NVDA), (Volume, NVDA)]
Index: []


In [42]:
df=pd.read_csv("/Users/thebeyonder/Downloads/NVDAm_Daily.csv",sep="\t")

In [46]:
df=df.rename(columns={"<DATE>":"Date","<OPEN>":"Open","<HIGH>":"High","<LOW>":"Low","<CLOSE>":"Close","<TICKVOL>":"Volume"})

In [782]:
df["diff_n_o"]=[None]+[np.log(df["Open"][i+1])-np.log(df["Open"][i]) for i in (range(len(df)-1))]
df["diff_n_h"]=[None]+[np.log(df["High"][i+1]-0.64)-np.log(df["High"][i]) for i in (range(len(df)-1))]
df["diff_n_c"]=[None]+[np.log(df["Close"][i+1])-np.log(df["Close"][i]) for i in (range(len(df)-1))]
df["diff_n_v"]=[None]+[np.log(df["Volume"][i+1])-np.log(df["Volume"][i]) for i in (range(len(df)-1))]
df["diff_n_l"]=[None]+[np.log(df["Low"][i+1])-np.log(df["Low"][i]) for i in (range(len(df)-1))]
df["diff_n_CO"]=[None]+[np.log(df["Close"][i+1])-np.log(df["Open"][i+1]) for i in (range(len(df)-1))]
df["diff_n_HO"]=[None]+[np.array(df["High"][i+1]-0.64)-np.array(df["Open"][i+1]) for i in (range(len(df)-1))]
data2=df.dropna()
Xl1=data2[["diff_n_o","diff_n_l","diff_n_h","diff_n_c","diff_n_v","diff_n_CO"]]

/var/folders/jm/hx406fwn5ql4cjk_zqccfpqw0000gn/T/ipykernel_80052/3791322620.py:2: RuntimeWarning: invalid value encountered in log
  df["diff_n_h"]=[None]+[np.log(df["High"][i+1]-0.64)-np.log(df["High"][i]) for i in (range(len(df)-1))]


In [783]:
df

,Open,High,Low,Close,Volume,diff_n_o,diff_n_h,diff_n_c,diff_n_v,diff_n_l,diff_n_CO,diff_n_HO,Predict
0,0.291833,0.296012,0.289279,0.295083,389396000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.292994,0.296708,0.290672,0.293922,353680000,0.003970,NaN,-0.003942,-0.096204,0.004804,0.003165,-0.636285,NaN
2,0.295548,0.297637,0.294155,0.297173,366168000,0.008679,NaN,0.010998,0.034700,0.011910,0.005484,-0.637911,NaN
3,0.299030,0.301816,0.295547,0.296940,342928000,0.011714,NaN,-0.000782,-0.065572,0.004724,-0.007012,-0.637214,NaN
4,0.298566,0.301584,0.294619,0.296244,314824000,-0.001554,NaN,-0.002348,-0.085507,-0.003147,-0.007806,-0.636982,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,124.800003,128.399994,116.699997,118.419998,818830900,-0.172997,-0.153591,-0.185946,1.249750,-0.195375,-0.052475,2.959991,-0.051798
2996,121.809998,129.000000,116.250000,128.990005,579666400,-0.024250,-0.000312,0.085497,-0.345425,-0.003863,0.057272,6.550002,-0.044110
2997,126.500000,126.889999,120.050003,123.699997,467120600,0.037780,-0.021548,-0.041876,-0.215865,0.032165,-0.022383,-0.250001,-0.037057
2998,123.099998,125.000000,118.099998,124.650002,392925500,-0.027245,-0.020140,0.007651,-0.172967,-0.016377,0.012513,1.260002,-0.075605


In [1183]:
timestep=10
X=[]
for i in range(len(Xl1)-timestep+1):
    c=np.array(Xl1[i:i+timestep])
    X.append(c)
X=np.array(X)

In [1184]:
Y=data2["diff_n_CO"].iloc[timestep:]

In [1185]:
Y.shape

(2989,)

In [1186]:
X_train=X[:-90]
X_test=X[-90:-1]
Y_train=Y[:-89]
Y_test=Y[-89:]

In [1040]:
X_train[-1]

array([[-2.96846740e-02,  2.67804377e-02, -1.49047292e-02,
         3.47861598e-02, -4.12184550e-01, -1.46382205e-02],
       [ 2.75535821e-02,  1.20783145e-02,  2.63965162e-02,
         1.51934432e-02, -2.07618842e-02,  3.02398967e-02],
       [ 1.45490322e-02,  2.32623058e-02,  6.87856217e-02,
         7.83476844e-02,  4.97957237e-01,  8.10339990e-02],
       [ 6.59715465e-02,  7.15702104e-02,  3.03425109e-02,
         1.89804006e-02, -1.84366588e-01,  8.54653674e-02],
       [ 1.89900822e-02,  1.90579943e-02, -6.89518509e-03,
        -3.35826672e-04, -4.31860649e-01,  1.91579942e-02],
       [-1.94180908e-02, -2.79376471e-02, -1.49494551e-02,
        -1.96716485e-02,  4.27630165e-02, -1.95037365e-02],
       [ 1.17468085e-02,  4.10143341e-03,  5.23256058e-03,
        -1.02423803e-02, -7.01200962e-02, -1.03280260e-02],
       [-1.94827935e-02, -1.41199538e-02, -9.30244770e-03,
        -1.93925888e-02,  2.91182554e-01, -4.14674233e-02],
       [ 1.25194263e-02,  3.49755450e-02,  1.651

In [1039]:
np.array(Y_train)[-2]

-0.011570711359166097

In [1035]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(2900, 10, 6) (89, 10, 6) (2900,) (89,)


In [1042]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    'AI_Close(T)_Open(T-1).keras',             # ชื่อไฟล์ที่จะบันทึกโมเดล
    monitor='val_acc',             # ติดตามการเปลี่ยนแปลงของ 'val_loss'
    save_best_only=True,            # บันทึกเฉพาะโมเดลที่ดีที่สุด
    save_weights_only=False,        # บันทึกทั้งโมเดล (ไม่ใช่แค่ weights)
    mode='max',                     # หาก 'monitor' เป็น 'val_loss' เราต้องการค่า 'min' (ค่าต่ำสุด)
    verbose=1                       # แสดงผลการบันทึกโมเดล
)
,callbacks=[checkpoint_callback]

In [1187]:
model.fit(X_train,Y_train,batch_size=32,epochs=200,validation_data=(X_test,Y_test))

Epoch 1/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - acc: 0.6778 - loss: 4.2630e-04 - mae: 0.0213 - val_acc: 0.6658 - val_loss: 6.9493e-04 - val_mae: 0.0254
Epoch 2/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - acc: 0.6977 - loss: 4.0372e-04 - mae: 0.0209 - val_acc: 0.7192 - val_loss: 7.6331e-04 - val_mae: 0.0261
Epoch 3/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - acc: 0.7494 - loss: 2.9230e-04 - mae: 0.0179 - val_acc: 0.6450 - val_loss: 7.3498e-04 - val_mae: 0.0260
Epoch 4/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - acc: 0.7491 - loss: 2.6072e-04 - mae: 0.0170 - val_acc: 0.6346 - val_loss: 7.6349e-04 - val_mae: 0.0263
Epoch 5/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - acc: 0.7379 - loss: 2.3258e-04 - mae: 0.0162 - val_acc: 0.7104 - val_loss: 7.8289e-04 - val_mae: 0.0273
Epoch 6/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - acc: 0.7822 - loss: 1.9569e-04 - mae: 0.0149 - val_acc: 0.6821 - val_loss: 7.7050e-04 - val_mae: 0.0267
Epoch 7/200
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step 

KeyboardInterrupt: 

In [145]:
np.array(Y_test)[0]

-0.013836663862268672

In [146]:
np.array(X_test)[0]

array([[ 2.75536537e-02,  1.20783862e-02,  2.63965878e-02,
         1.51935148e-02, -2.07618842e-02,  2.68631462e-03],
       [ 1.45490322e-02,  2.32623058e-02,  6.87856217e-02,
         7.83476844e-02,  4.97957237e-01,  6.64849668e-02],
       [ 6.59715465e-02,  7.15702104e-02,  3.03425109e-02,
         1.89804006e-02, -1.84366588e-01,  1.94938209e-02],
       [ 1.89900822e-02,  1.90579943e-02, -6.89518509e-03,
        -3.35826672e-04, -4.31860649e-01,  1.67912025e-04],
       [-1.94180908e-02, -2.79376471e-02, -1.49494551e-02,
        -1.96716485e-02,  4.27630165e-02, -8.56457272e-05],
       [ 1.17468085e-02,  4.10143341e-03,  5.23256058e-03,
        -1.02423803e-02, -7.01200962e-02, -2.20748345e-02],
       [-1.94827935e-02, -1.41199538e-02, -9.30244770e-03,
        -1.93925888e-02,  2.91182554e-01, -2.19846298e-02],
       [ 1.25194263e-02,  3.49755450e-02,  1.65154073e-02,
         3.89254923e-02, -5.57010336e-02,  4.42143625e-03],
       [-2.47429946e-03, -1.59907106e-02, -8.729

In [199]:
len(X_train)

2900

In [197]:
np.array(df["diff_n_h"][-89:])==np.array(Y_test)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [214]:
## calculate profit from the model
pred=model.predict(X_test)
profit=[]
for i in range(len(X_test)):
    if pred[i][0]>0:
        profit.append(np.array(df["High"])[-(88-i)]-np.array(df["Open"])[-(89-i)])
    else:
        print("Sell")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell


In [211]:
pred[i][0]

-0.008021899

In [218]:
profit

[8.41943199453543,
 5.649610689654708,
 -2.7698121842949206,
 -2.389829192210726,
 4.11971659675217,
 5.69960647716448,
 8.489410606756138,
 4.259714984073042,
 0.8899343038750658,
 3.8697276601940587,
 5.5896381718447685,
 2.099858400087271,
 -1.2499201087975393,
 -0.439974762792815,
 6.289566876538345,
 -0.47998084107297245,
 -0.6799483396797257,
 2.20984506992491,
 -1.929855040516344,
 -0.28998994968870306,
 3.1597619214169015,
 9.039381576713367,
 5.969586170327034,
 3.3797621494882435,
 0.08000088407754902,
 0.9699295063833802,
 2.549825276651518,
 -0.06999917721532256,
 -3.3197673908569243,
 7.629478793236075,
 6.239559272530329,
 5.479626340152919,
 4.339714740632019,
 3.669741007549419,
 1.709879142179176,
 7.529480599336381,
 4.5497715690091525,
 0.589996337890625,
 -4.6500091552734375,
 1.160003662109375,
 2.5200042724609375,
 -4.540008544921875,
 0.1699981689453125,
 5.6199951171875,
 0.850006103515625,
 -0.67999267578125,
 1.720001220703125,
 3.2400054931640625,
 0.85000610

In [1244]:
df[-10:]

,Open,High,Low,Close,Volume,diff_n_o,diff_n_h,diff_n_c,diff_n_v,diff_n_l,diff_n_CO,Predict1,Predict3,Predict4
2990,136.690002,138.500000,135.460007,137.710007,201188800,-0.014165,-0.001803,0.030524,-0.039217,0.014650,-0.006731,-0.025967,-0.007938,-0.020281
2991,139.160004,141.830002,137.089996,140.830002,197749000,0.017909,0.023759,0.022403,-0.017245,0.011961,0.029838,0.027069,0.004026,0.008349
2992,144.660004,147.789993,143.669998,147.070007,237651400,0.038762,0.041163,0.043355,0.183806,0.046881,0.055284,0.018628,0.029927,0.024383
2993,145.050003,147.229996,143.720001,147.220001,155915500,0.002692,-0.003796,0.001019,-0.421491,0.000348,0.017542,0.040416,0.037739,0.023775
2994,148.369995,148.970001,141.880005,142.619995,234657600,0.022631,0.011749,-0.031744,0.408813,-0.012885,-0.016895,0.008457,0.003122,0.001132
2995,124.800003,128.399994,116.699997,118.419998,818830900,-0.172997,-0.148595,-0.185946,1.249750,-0.195375,-0.225472,-0.030561,-0.027245,-0.035415
2996,121.809998,129.000000,116.250000,128.990005,579666400,-0.024250,0.004662,0.085497,-0.345425,-0.003863,0.033022,-0.039500,-0.091228,-0.080644
2997,126.500000,126.889999,120.050003,123.699997,467120600,0.037780,-0.016492,-0.041876,-0.215865,0.032165,0.015397,0.003484,-0.025583,0.003184
2998,123.099998,125.000000,118.099998,124.650002,392925500,-0.027245,-0.015007,0.007651,-0.172967,-0.016377,-0.014732,-0.005678,-0.003485,-0.003002
2999,123.779999,127.849998,119.190002,120.070000,388161100,0.005509,0.022544,-0.037435,-0.012200,0.009187,-0.024922,-0.007996,0.017470,0.005748


In [1225]:
X[-1]

array([[-1.41650350e-02,  1.46498335e-02, -1.80342700e-03,
         3.05243337e-02, -3.92171191e-02, -6.73056751e-03],
       [ 1.79087706e-02,  1.19611744e-02,  2.37588446e-02,
         2.24034287e-02, -1.72452209e-02,  2.98378962e-02],
       [ 3.87618101e-02,  4.68813725e-02,  4.11631316e-02,
         4.33552115e-02,  1.83806344e-01,  5.52843371e-02],
       [ 2.69234487e-03,  3.47980461e-04, -3.79634094e-03,
         1.01936121e-03, -4.21490700e-01,  1.75418882e-02],
       [ 2.26305894e-02, -1.28853064e-02,  1.17489907e-02,
        -3.17443586e-02,  4.08813237e-01, -1.68948152e-02],
       [-1.72996641e-01, -1.95375151e-01, -1.48594608e-01,
        -1.85946105e-01,  1.24975016e+00, -2.25471509e-01],
       [-2.42500467e-02, -3.86346867e-03,  4.66206064e-03,
         8.54973129e-02, -3.45424826e-01,  3.30224442e-02],
       [ 3.77798745e-02,  3.21653036e-02, -1.64918398e-02,
        -4.18756698e-02, -2.15865297e-01,  1.53968211e-02],
       [-2.72452874e-02, -1.63766378e-02, -1.500

In [1308]:
m4=tf.keras.models.load_model('AI4.keras')
m1=tf.keras.models.load_model('AI_Close(T)_Open(T-1).keras')
m3=tf.keras.models.load_model('AI3.keras')


In [1309]:
pred1=m1.predict(X[:-1])
pred3=m3.predict(X[:-1])
pred4=m4.predict(X[:-1])

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [1310]:
pred1[-1]

array([-0.00799608], dtype=float32)

In [1311]:
## Note ค่า prediction ที่ได้คือผลการทำนายของวันพรุ่งนี้ซึ่งอยู่ในเเถวเดียวกับค่าจริงในวันพรุ่งนี้ โดยใช้ข้อมูลถึงวันนี้
df["Predict1"] = [None] * (len(df) - len(pred1)) + list(pred1.flatten())
df["Predict3"] = [None] * (len(df) - len(pred3)) + list(pred3.flatten())
df["Predict4"] = [None] * (len(df) - len(pred4)) + list(pred4.flatten())

In [1303]:
### ถ้าผลการทำนายราคาสูงสุดวันนี้บอกว่าราคาสูงสุดพรุ่งนี้จะสูงกว่าวันนี้ เเละราคาเปิดวันพรุ่งนนี้ต่ำกว่าราคาสูงสุดวันนี้ให้ซื้อที่ราคาเปิดวันพรุ่งนี้ เเละตั้งขายที่ราคาสูงสุดวันนี้
### หากบังเอิญโมเดลทำนายผิดจะขายที่ราคาปิด หรือตามเห็นสมควรเเทน
### หากผลการทำนายติดลบจะไม่ซื้อ
profit2=[]
for i in range(10,len(df)-1):
    if   df["Predict4"][i]>0:## i คือค่า Predict ของเวลา i-1 
        if df["High"][i]>df["Open"][i+1]:
            if df["High"][i+1]>df["High"][i]:
                profit2.append(np.array(df["High"])[i]-np.array(df["Open"])[i+1])
            else:
                profit2.append(np.array(df["Close"])[i+1]-np.array(df["Open"])[i+1])
        else:
            profit2.append(np.array(df["Close"])[i+1]-np.array(df["Open"])[i+1])
    else:
        profit2.append(0)

In [1314]:
### หากโมเดลทำนายว่าราคาปิดวันนี้จะสูงกว่าราคาเปิดเมื่อวาน จะซื้อที่ราคาเปิดเมื่อวานเเละขายที่ราคาปิดวันนี้
profit2=[]
for i in range(10,len(df)-1):
    if   df["Predict1"][i+1]>0:### i+1 คือผลpredict ที่เวลา i 
        profit2.append(np.array(df["Close"])[i+1]-np.array(df["Open"])[i])
    else:
        profit2.append(0)

In [1315]:
min(profit2)

-8.449996948242188

In [1316]:
np.sum(profit2)

823.9731171439074

In [1317]:
profit2[-30:]

[0,
 -0.17999267578125,
 0,
 2.94000244140625,
 9.860000610351562,
 3.94000244140625,
 -0.07000732421875,
 0,
 0,
 -0.540008544921875,
 0.279998779296875,
 8.470001220703125,
 9.419998168945312,
 -8.449996948242188,
 0,
 -6.6699981689453125,
 0,
 0,
 0.19000244140625,
 -0.079986572265625,
 0,
 4.1399993896484375,
 7.910003662109375,
 2.55999755859375,
 -2.4300079345703125,
 0,
 0,
 1.8899993896484375,
 0,
 0]

In [1318]:
np.sum(np.array(profit2)>0)/np.sum(np.array(profit2)!=0)

0.8713578889499725

In [1272]:
print(classification_report(df["diff_n_h"][10:]>0,df["Predict4"][10:]>0))

              precision    recall  f1-score   support

       False       0.81      0.82      0.81      1419
        True       0.84      0.82      0.83      1571

    accuracy                           0.82      2990
   macro avg       0.82      0.82      0.82      2990
weighted avg       0.82      0.82      0.82      2990



In [869]:
model.predict(X[-2:-1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


array([[0.01757547]], dtype=float32)

In [1084]:
df[:20]

,Open,High,Low,Close,Volume,diff_n_o,diff_n_h,diff_n_c,diff_n_v,diff_n_l,diff_n_CO,Predict1,Predict3,Predict4
0,0.291833,0.296012,0.289279,0.295083,389396000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.292994,0.296708,0.290672,0.293922,353680000,3.969626e-03,0.002350,-0.003942,-0.096204,0.004804,0.007134,NaN,NaN,NaN
2,0.295547,0.297637,0.294154,0.297173,366168000,8.678550e-03,0.003125,0.010998,0.034700,0.011910,0.014162,NaN,NaN,NaN
3,0.299030,0.301816,0.295547,0.296940,342928000,1.171416e-02,0.013943,-0.000782,-0.065572,0.004724,0.004702,NaN,NaN,NaN
4,0.298566,0.301584,0.294619,0.296244,314824000,-1.553603e-03,-0.000769,-0.002348,-0.085507,-0.003147,-0.009360,NaN,NaN,NaN
5,0.296940,0.299030,0.295547,0.297637,246464000,-5.458342e-03,-0.008504,0.004691,-0.244798,0.003147,-0.003116,NaN,NaN,NaN
6,0.294619,0.296012,0.292297,0.294851,299748000,-7.849210e-03,-0.010144,-0.009404,0.195726,-0.011058,-0.007062,NaN,NaN,NaN
7,0.294387,0.296708,0.291601,0.295780,310856000,-7.886051e-04,0.002350,0.003144,0.036388,-0.002386,0.003932,NaN,NaN,NaN
8,0.295780,0.300887,0.292994,0.295780,485840000,4.720734e-03,0.013986,0.000000,0.446550,0.004766,0.004721,NaN,NaN,NaN
9,0.297637,0.298566,0.294154,0.296012,383304000,6.259858e-03,-0.007746,0.000785,-0.237051,0.003954,0.000785,NaN,NaN,NaN


In [307]:
spread=[]
for i in range(1,len(df)):
    if df["Predict"][i] > 0:
        spread.append(df["High"][i-1]-df["Open"][i]-0.64)
    else:
        print("Sell")

Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell
Sell


In [790]:
pred[-20:]

array([[-1.13206767e-02],
       [ 2.83559356e-02],
       [ 7.91198760e-03],
       [ 1.50016323e-03],
       [-8.64018574e-02],
       [ 1.83829665e-03],
       [-6.37024641e-07],
       [-2.44047381e-02],
       [ 4.31827083e-03],
       [ 8.57927464e-03],
       [-2.27941908e-02],
       [ 5.57442941e-03],
       [ 1.05525479e-02],
       [ 2.83757839e-02],
       [ 1.23723205e-02],
       [-2.05297843e-02],
       [-2.35746205e-02],
       [-1.61340274e-02],
       [-5.40531427e-03],
       [-7.44154304e-03]], dtype=float32)

In [1146]:
data5 = yf.download(symbol, period="11d", interval="1d")

df5=pd.DataFrame()
df5["Open"]=np.array(data5["Open"]["NVDA"])
df5["High"]=np.array(data5["High"]["NVDA"])
df5["Low"]=np.array(data5["Low"]["NVDA"])
df5["Close"]=np.array(data5["Close"]["NVDA"])
df5["Volume"]=np.array(data5["Volume"]["NVDA"])

df5["diff_n_o"]=[None]+[np.log(df5["Open"][i+1])-np.log(df5["Open"][i]) for i in (range(len(df5)-1))]
df5["diff_n_h"]=[None]+[np.log(df5["High"][i+1])-np.log(df5["High"][i]) for i in (range(len(df5)-1))]
df5["diff_n_c"]=[None]+[np.log(df5["Close"][i+1])-np.log(df5["Close"][i]) for i in (range(len(df5)-1))]
df5["diff_n_v"]=[None]+[np.log(df5["Volume"][i+1])-np.log(df5["Volume"][i]) for i in (range(len(df5)-1))]
df5["diff_n_l"]=[None]+[np.log(df5["Low"][i+1])-np.log(df5["Low"][i]) for i in (range(len(df5)-1))]
df5["diff_n_CO"]=[None]+[np.log(df5["Close"][i+1])-np.log(df5["Open"][i]) for i in (range(len(df5)-1))]
data5=df5.dropna()
Xl5=data5[["diff_n_o","diff_n_l","diff_n_h","diff_n_c","diff_n_v","diff_n_CO"]]

[*********************100%***********************]  1 of 1 completed


In [1147]:
m1.predict(Xl5.values[np.newaxis,-10:,:])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[-0.0021877]], dtype=float32)

In [1103]:
dm = yf.download(symbol, period="2d", interval="1m")

[*********************100%***********************]  1 of 1 completed


In [1104]:
dm

Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Datetime,,,,,
2025-01-30 14:30:00+00:00,123.110001,124.150002,123.000000,123.080002,16459594
2025-01-30 14:31:00+00:00,123.315002,123.669998,122.959999,123.105003,1425718
2025-01-30 14:32:00+00:00,122.649902,123.690002,122.500000,123.330002,1751337
2025-01-30 14:33:00+00:00,122.425598,122.900002,122.250000,122.650002,1613022
2025-01-30 14:34:00+00:00,122.214996,122.500000,121.919998,122.447197,1725838
...,...,...,...,...,...
2025-01-31 20:55:00+00:00,120.179901,120.250000,119.989998,120.074997,1134160
2025-01-31 20:56:00+00:00,120.165001,120.279999,120.010002,120.190002,1802534


In [1096]:
data

Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2025-01-17,137.710007,138.500000,135.460007,136.690002,201188800
2025-01-21,140.830002,141.830002,137.089996,139.160004,197749000
2025-01-22,147.070007,147.789993,143.669998,144.660004,237651400
2025-01-23,147.220001,147.229996,143.720001,145.050003,155915500
2025-01-24,142.619995,148.970001,141.880005,148.369995,234657600
2025-01-27,118.419998,128.399994,116.699997,124.800003,818830900
2025-01-28,128.990005,129.000000,116.250000,121.809998,579666400
2025-01-29,123.699997,126.889999,120.050003,126.500000,467120600


In [1105]:
np.sum(dm["Volume"]["NVDA"][:-390])

342284732

In [1106]:
len(dm["Volume"]["NVDA"][:-390])

390

In [1120]:
model.predict(Xl1.values[np.newaxis])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


array([[0.0002049]], dtype=float32)

In [1113]:
Xl1.values.shape

(9, 6)